In [ ]:
import pandas as pd
from pymongo import MongoClient

In [13]:
mixer_contracts = [
    ('0xA160cdAB225685dA1d56aa342Ad8841c3b53f291', '100 Eth'),
    ('0x910Cbd523D972eb0a6f4cAe4618aD62622b39DbF', '10 Eth'),
    ('0x47CE0C6eD5B0Ce3d3A51fdb1C52DC66a7c3c2936', '1 Eth'),
    ('0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc', '0.1 Eth'),
    ('0x23773E65ed146A459791799d01336DB287f25334', '100k Dai'),
    ('0x07687e702b410Fa43f4cB4Af7FA097918ffD2730', '10k Dai'),
    ('0xFD8610d20aA15b7B2E3Be39B396a1bC3516c7144', '1k Dai'),
    ('0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3', '100 Dai'),
    ('0xbB93e510BbCD0B7beb5A853875f9eC60275CF498', '10 wBTC'),
    ('0x610B717796ad172B316836AC95a2ffad065CeaB4', '1 wBTC'),
    ('0x178169B423a011fff22B9e3F3abeA13414dDD0F1', '0.1 wBTC'),
    ('0xD691F27f38B395864Ea86CfC7253969B409c362d', '10k USDC'),
    ('0xd96f2B1c14Db8458374d9Aca76E26c3D18364307', '100 USDC'),
    ('0x4736dCf1b7A3d580672CcE6E7c65cd5cc9cFBa9D', '1k USDC'),
    ('0x9AD122c22B14202B4490eDAf288FDb3C7cb3ff5E', '100k USDT'),
    ('0xF67721A2D8F736E75a49FdD7FAd2e31D8676542a', '10k USDT'),
    ('0x0836222F2B2B24A3F36f98668Ed8F0B38D1a872f', '1k USDT'),
    ('0x169AD27A470D064DEDE56a2D3ff727986b15D52B', '100 USDT'),
    ('0xD21be7248e0197Ee08E0c20D4a96DEBdaC3D20Af', '5M cDai'),
    ('0x2717c5e28cf931547b621a5dddb772ab6a35b701', '500k cDai'),
    ('0xBA214C1c1928a32Bffe790263E38B4Af9bFCD659', '50k cDai'),
    ('0x22aaA7720ddd5388A3c0A3333430953C68f1849b', '5k cDai'),
    ('0x1356c899D8C9467C7f71C195612F8A395aBf2f0a', '50k cUSDC'),
    ('0xaEaaC358560e11f52454D997AAFF2c5731B6f8a6', '5k cUSDC')
]

df = pd.DataFrame(mixer_contracts, columns=['contract_address', 'pool'])
df['contract_address'] = df['contract_address'].str.lower()
df.set_index('contract_address', inplace=True)
df.to_csv('../data/tornado-pools.csv')
df

,pool
contract_address,
0xa160cdab225685da1d56aa342ad8841c3b53f291,100 Eth
0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,10 Eth
0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1 Eth
0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
0x23773e65ed146a459791799d01336db287f25334,100k Dai
0x07687e702b410fa43f4cb4af7fa097918ffd2730,10k Dai
0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,1k Dai
0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3,100 Dai
0xbb93e510bbcd0b7beb5a853875f9ec60275cf498,10 wBTC


In [24]:
# Connect to db collections
client = MongoClient('mongodb://localhost:27017/')
db = client['address-clustering']
transfers = db['transfers']
transactions = db['transactions']
tornado_pools = pd.read_csv('../data/tornado-pools.csv')['contract_address'].tolist()

# Query
tornado_transactions = pd.DataFrame(list(transactions.find({"$or": [{"from": {"$in": tornado_pools}}, {"to": {"$in": tornado_pools}}]})))

# Dataframe operations
tornado_transactions['tokenName'] = 'Ether'
tornado_transactions['tokenType'] = 'native'
tornado_transactions['isSet'] = 'from'
tornado_transactions['userAddress'] = tornado_transactions['from']
tornado_transactions['contractAddress'] = tornado_transactions['to']
tornado_transfers = pd.DataFrame(list(transfers.find({"$or": [{"from": {"$in": tornado_pools}}, {"to": {"$in": tornado_pools}}]})))
df = pd.concat([tornado_transactions,tornado_transfers])
df.drop(labels=['_id','tokenID'], axis=1, inplace=True)
df

,hash,from,to,timeStamp,nonce,value,gasPrice,input,gasUsed,functionName,chainName,tokenName,tokenType,isSet,userAddress,contractAddress
0,0xab9112a0bfd7ee489f32ad2eace95cd9111da9091008...,0x007f44362400de9f364efa919ff84c6ea9e210d5,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1597667681,171.0,1.000000e+17,112000000000,0xb214faa52606de7c18da92872e783b4cf27c757334ca...,966887,deposit(bytes32 _id),Ethereum,Ether,native,from,0x007f44362400de9f364efa919ff84c6ea9e210d5,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
1,0xb96895d541a818232300808230ff202f8e1217609f10...,0x1dd5df3b2d78271039f3dc39eb9350ecc31ca9dc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1605348173,39.0,1.000000e+17,23000000000,0xb214faa50bf6fe5e24e3223b7e2cec17a028597694c1...,978703,deposit(bytes32 _id),Ethereum,Ether,native,from,0x1dd5df3b2d78271039f3dc39eb9350ecc31ca9dc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
2,0x282777a6664cf1d9d3f1695bed549f7a62ec57411350...,0x1ec594a869dc67d78fabc33963d3c6a0ab017dd6,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1602676051,827.0,1.000000e+17,33000000000,0xb214faa512dd5f64b38ff2414a7e7cf5219e5ff98ec6...,978703,deposit(bytes32 _id),Ethereum,Ether,native,from,0x1ec594a869dc67d78fabc33963d3c6a0ab017dd6,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
3,0xc880becfb7f3a939765f1cb650cd7b624cdb0a046a22...,0x2fef65e4d69a38bf0dd074079f367cdf176ec0de,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1603804997,7601.0,1.000000e+17,45000000000,0xb214faa50205af153fb0c6e1818a2e396450ed9baf14...,972795,deposit(bytes32 _id),Ethereum,Ether,native,from,0x2fef65e4d69a38bf0dd074079f367cdf176ec0de,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
4,0xd9eeeec262897f88c4323013ca927cb3cb94e7f22347...,0x30602250c5f1fcba5407e99b1dfaab992ea4ffd2,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1583900510,4830.0,1.000000e+17,13000000000,0xb214faa5068a8346b266fcdfd7a9830a2409eac71681...,984611,deposit(bytes32 _id),Ethereum,Ether,native,from,0x30602250c5f1fcba5407e99b1dfaab992ea4ffd2,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0x5607c93ba2fc0814327af6f53b53923bcc747edc7b2b...,0x169ad27a470d064dede56a2d3ff727986b15d52b,0x3f0500b79c099dfe2638d0faf1c03f56b90d12d1,1602750994,2787.0,7.718709e+01,80000000000,NaN,361300,NaN,Ethereum,Tether USD,20,to,0x3f0500b79c099dfe2638d0faf1c03f56b90d12d1,0xdac17f958d2ee523a2206206994597c13d831ec7
5,0xdb5ea6e7a175dda3b9359910aaa5ebf3f70b5e2ab253...,0x178169b423a011fff22b9e3f3abea13414ddd0f1,0x0c40bff248ed5d99d294ceca8826c7924f745134,1619556022,3439.0,9.831973e-02,58650000000,NaN,399260,NaN,Ethereum,Wrapped BTC,20,to,0x0c40bff248ed5d99d294ceca8826c7924f745134,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
6,0x7bce45c79f340555a9c7e66116875cb56ad065a24488...,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0x7a218b62ae9e23aac788979ead8be51258a3cbc5,1621903058,154.0,1.000000e+03,46000000000,NaN,395784,NaN,Ethereum,Dai Stablecoin,20,to,0x7a218b62ae9e23aac788979ead8be51258a3cbc5,0x6b175474e89094c44da98b954eedeac495271d0f
7,0x9b7a76858cc9ca49de2cb273e4a2e635e4b652019acf...,0x62051bfd3a4f7039a849142e6e5ea172cbda5949,0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3,1598285598,212.0,1.000000e+02,128000000000,NaN,1003512,NaN,Ethereum,Dai Stablecoin,20,from,0x62051bfd3a4f7039a849142e6e5ea172cbda5949,0x6b175474e89094c44da98b954eedeac495271d0f
